# ノック41(自身の解答)

ノック41 - 50では、決定木を用いて退会会員の予測をしていく。

In [1]:
import pandas as pd

In [2]:
# データの読み込み

costomer = pd.read_csv("customer_join.csv")
costomer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months = pd.read_csv("use_log_months.csv")
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


今回は1カ月前のデータから当月の退会予測を導き出す。  
まずは、会員・月毎の利用回数を集計。

In [4]:
uselog_months2 = uselog_months.pivot_table(index = "customer_id", columns = "年月", values = "count").reset_index()
uselog_months2

年月,customer_id,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903
0,AS002855,4.0,5.0,5.0,5.0,3.0,7.0,3.0,5.0,2.0,6.0,5.0,4.0
1,AS008805,NaN,NaN,8.0,7.0,5.0,2.0,2.0,3.0,5.0,1.0,6.0,1.0
2,AS009013,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AS009373,3.0,4.0,4.0,7.0,6.0,6.0,5.0,4.0,4.0,7.0,6.0,5.0
4,AS015233,NaN,7.0,7.0,5.0,11.0,9.0,7.0,7.0,9.0,9.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,TS995853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0
4188,TS998593,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,7.0,8.0,7.0,8.0
4189,TS999079,7.0,5.0,6.0,2.0,3.0,4.0,6.0,6.0,9.0,6.0,2.0,3.0
4190,TS999231,6.0,4.0,1.0,5.0,3.0,4.0,5.0,5.0,8.0,3.0,6.0,6.0


次に、当月の利用回数とその前の月の利用回数を集計していく。

In [5]:
# uselog_months2を月毎に分ける。また、後程結合するために列名を"month"に変更する。

uselog_months_201804 = uselog_months2[["customer_id", 201804]].reset_index(drop = True)
uselog_months_201804.rename(columns = {201804 : "month"}, inplace = True)

uselog_months_201805 = uselog_months2[["customer_id", 201805]].reset_index(drop = True)
uselog_months_201805.rename(columns = {201805 : "month"}, inplace = True)

uselog_months_201806 = uselog_months2[["customer_id", 201806]].reset_index(drop = True)
uselog_months_201806.rename(columns = {201806 : "month"}, inplace = True)

uselog_months_201807 = uselog_months2[["customer_id", 201807]].reset_index(drop = True)
uselog_months_201807.rename(columns = {201807 : "month"}, inplace = True)

uselog_months_201808 = uselog_months2[["customer_id", 201808]].reset_index(drop = True)
uselog_months_201808.rename(columns = {201808 : "month"}, inplace = True)

uselog_months_201809 = uselog_months2[["customer_id", 201809]].reset_index(drop = True)
uselog_months_201809.rename(columns = {201809 : "month"}, inplace = True)

uselog_months_201810 = uselog_months2[["customer_id", 201810]].reset_index(drop = True)
uselog_months_201810.rename(columns = {201810 : "month"}, inplace = True)

uselog_months_201811 = uselog_months2[["customer_id", 201811]].reset_index(drop = True)
uselog_months_201811.rename(columns = {201811 : "month"}, inplace = True)

uselog_months_201812 = uselog_months2[["customer_id", 201812]].reset_index(drop = True)
uselog_months_201812.rename(columns = {201812 : "month"}, inplace = True)

uselog_months_201901 = uselog_months2[["customer_id", 201901]].reset_index(drop = True)
uselog_months_201901.rename(columns = {201901 : "month"}, inplace = True)

uselog_months_201902 = uselog_months2[["customer_id", 201902]].reset_index(drop = True)
uselog_months_201902.rename(columns = {201902 : "month"}, inplace = True)

uselog_months_201903 = uselog_months2[["customer_id", 201903]].reset_index(drop = True)
uselog_months_201903.rename(columns = {201903 : "month"}, inplace = True)

In [6]:
# 当月の利用回数をまとめる(5月以降)

uselog_months_first = pd.concat([uselog_months_201805, uselog_months_201806,uselog_months_201807,uselog_months_201808, uselog_months_201809, uselog_months_201810, uselog_months_201811, uselog_months_201812, uselog_months_201901, uselog_months_201902, uselog_months_201903],axis = 0).reset_index(drop = True)
uselog_months_first.rename(columns = {"month" : "count_0"},inplace = True)
uselog_months_first

年月,customer_id,count_0
0,AS002855,5.0
1,AS008805,NaN
2,AS009013,NaN
3,AS009373,4.0
4,AS015233,7.0
...,...,...
46107,TS995853,8.0
46108,TS998593,8.0
46109,TS999079,3.0
46110,TS999231,6.0


In [7]:
# 前月の利用回数をまとめる(2019年2月以前)

uselog_months_second = pd.concat([uselog_months_201804,uselog_months_201805, uselog_months_201806,uselog_months_201807,uselog_months_201808, uselog_months_201809, uselog_months_201810, uselog_months_201811, uselog_months_201812, uselog_months_201901, uselog_months_201902],axis = 0).reset_index(drop = True)
uselog_months_second.rename(columns = {"month" : "count_1"},inplace = True)
del uselog_months_second["customer_id"]
uselog_months_second

年月,count_1
0,4.0
1,NaN
2,2.0
3,3.0
4,NaN
...,...
46107,11.0
46108,7.0
46109,2.0
46110,6.0


In [13]:
# 上記2つのファイルを結合

uselog_months3 = pd.concat([uselog_months_first, uselog_months_second], axis = 1)
uselog_months3

年月,customer_id,count_0,count_1
0,AS002855,5.0,4.0
1,AS008805,NaN,NaN
2,AS009013,NaN,2.0
3,AS009373,4.0,3.0
4,AS015233,7.0,NaN
...,...,...,...
46107,TS995853,8.0,11.0
46108,TS998593,8.0,7.0
46109,TS999079,3.0,2.0
46110,TS999231,6.0,6.0


In [14]:
# count_0(当月)が欠損値だと意味が無いので、そちらを省く

uselog_months3.loc[~uselog_months3["count_0"].isnull(), :]

年月,customer_id,count_0,count_1
0,AS002855,5.0,4.0
3,AS009373,4.0,3.0
4,AS015233,7.0,NaN
5,AS015315,3.0,6.0
6,AS015739,5.0,7.0
...,...,...,...
46107,TS995853,8.0,11.0
46108,TS998593,8.0,7.0
46109,TS999079,3.0,2.0
46110,TS999231,6.0,6.0


以上で、当月の利用回数とその前の月の利用回数が集計された。